# Two Disks in a Rectangular Box: mean first event times.

Here are the simulations and calculations, numerical vs. analytical, of the mean first times and their distributions, for a series of interesting events, namely, hopping, crashing and wall-hitting.

The external file contains all the routines for initial conditions and dynamic rules.



In [1]:
include("discs_in_box.jl")

dynamics

In [2]:
#auxiliary libs for ploting
using Plots, Interact
gr()

Plots.GRBackend()

In [55]:
# A small test. w is widht, h height, r is radius for both discs.

w, h = 1.5, 1
r = 0.24

#= dynamics returns a tuple of results, in which we have the timestamp of an event, the position of the center
oof the discs, the velocites of each disc and the collition type according to following code:
+1/-1 -> disc one hits right/left wall
+2/-2 -> disc one hits top/bottom wall
+3/-3  -> disc two hits right/left wall
+4/-4 -> disc two hits top/bottom wall.
5 -> discs collide.
=#
@time times, positions, velocities, collision_types = dynamics(w, h, r, 1000000);

  2.814708 seconds (13.84 M allocations: 1005.737 MiB, 74.22% gc time)


In [4]:
showall(norm.(velocities)[1:100]) #Sanity Check

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

In [5]:
showall(collision_types[1:100])#Sanity Check

[2, 4, -2, -3, 5, -3, 4, -2, 5, -2, -3, 4, 2, 1, 5, -2, 5, -3, 4, 5, 1, 2, 5, -4, 5, -3, 2, 5, 1, -3, 5, -3, -4, 2, 5, 2, -4, 5, -4, 2, -3, 1, 4, 5, -2, 5, 4, -3, 2, -2, 1, 5, 1, -4, 5, 2, 5, 1, -3, -2, 5, 1, 4, -3, 5, -3, -2, 5, 1, 4, 5, -3, 5, 1, 5, 1, -3, 2, -4, 5, -4, 2, 5, -4, 2, 5, 2, -4, 5, -4, 2, 5, 2, -4, 5, -4, 2, 5, 2, -4]

The function `dynamics` returns data on the collisions that have occurred.

In [6]:
function draw_disc!(x, y, r)
    θs = [0:0.1:2π; 0]
    xs = [x + r*cos(θ) for θ in θs]
    ys = [y + r*sin(θ) for θ in θs]
    
    plot!(xs, ys)
end

draw_disc! (generic function with 1 method)

In [7]:
@manipulate for i in slider(1:length(positions), value=1)
    x1, y1, x2, y2 = positions[i]
    
    a = w/2
    b = h/2
    
    plot([-a, a, a, -a, -a], [-b, -b, b, b, -b], aspectratio=1, leg=false, xlim=(-0.75,0.75), ylim=(-0.5,0.5))
    draw_disc!(x1, y1, r) 
    draw_disc!(x2, y2, r) 
end

Interact.Slider{Int64}(1: "input" = 1 Int64 , "", 1, 1:1000000, "horizontal", true, "d", true)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -0.75 
 
 
 -0.50 
 
 
 -0.25 
 
 
 0.00 
 
 
 0.25 
 
 
 0.50 
 
 
 0.75 
 
 
 -0.50 
 
 
 -0.25 
 
 
 0.00 
 
 
 0.25 
 
 
 0.50 
 
 
<polyline clip-path="url(#clip1802)" style="stroke:#e26f46; stroke-width:1; stroke-opacity:1; fill:none" points="
 530.57,101.151 530.142,92.6123 528.865,84.1586 526.75,75.8746 523.818,67.8433 520.099,60.1447 515.63,52.8558 510.456,46.0495 504.628,39.7937 498.205,34.151 
 491.25,29.1778 483.834,24.9237 476.03,21.4312 467.917,18.7353 459.574,16.8629 451.087,15.8326 442.539,15.6548 434.016,16.3313 425.603,17.8552 417.385,20.2114 
 409.442,23.3764 401.856,27.3184 394.7,31.9982 388.048,37.3689 381.965,43.377 376.512,49.9622 371.744,57.059 367.709,64.5963 364.445,72.4988 361.988,80.6877 
 360.36,89.081 359.578,97.5949 359.649,106.144 360.574,114.644 362.343,123.009 364.939,131.155 368.334,139.002 372.496,146.47 377.383,153.485 382.945,159.978 
 389.128,165.883 395.87,171.141 403.103,175.7 410.755,179.514 418.75,182.545 427.007,184.763 435.444,186.145 443.977,186.678 452.521,186.356 460.99,185.184 
 469.299,183.171 477.366,180.339 485.11,176.716 492.454,172.338 499.324,167.248 505.651,161.498 511.373,155.146 516.432,148.253 520.778,140.89 524.367,133.13 
 527.163,125.051 529.139,116.732 530.274,108.258 530.57,101.151 
 "/>
<polyline clip-path="url(#clip1802)" style="stroke:#3da44d; stroke-width:1; stroke-opacity:1; fill:none" points="
 309.398,124.954 308.971,116.415 307.693,107.961 305.578,99.6773 302.646,91.6459 298.927,83.9473 294.458,76.6585 289.284,69.8522 283.456,63.5964 277.033,57.9537 
 270.079,52.9805 262.662,48.7264 254.859,45.2339 246.745,42.538 238.403,40.6655 229.915,39.6353 221.367,39.4575 212.844,40.1339 204.432,41.6579 196.213,44.0141 
 188.271,47.1791 180.684,51.1211 173.529,55.8009 166.876,61.1716 160.793,67.1796 155.341,73.7649 150.573,80.8617 146.537,88.399 143.274,96.3015 140.816,104.49 
 139.188,112.884 138.406,121.398 138.478,129.947 139.403,138.447 141.172,146.811 143.767,154.958 147.162,162.804 151.324,170.273 156.211,177.288 161.774,183.781 
 167.957,189.686 174.698,194.944 181.931,199.503 189.583,203.317 197.578,206.348 205.835,208.565 214.272,209.948 222.805,210.481 231.349,210.159 239.818,208.986 
 248.127,206.974 256.195,204.142 263.939,200.519 271.282,196.14 278.152,191.051 284.48,185.301 290.201,178.948 295.26,172.056 299.606,164.693 303.195,156.933 
 305.991,148.853 307.967,140.535 309.102,132.061 309.398,124.954 
 "/>

# Hopping

The data from the simulation directly gives disc and wall collision times.
The other times that we require are hopping times, both horizontal, when $x_1 = x_2$, and vertical, when $y_1 = y_2$. 

Vertical hops can be detected from the data when $y_2 - y_1$ changes sign between two consecutive collisions.
Using the relative velocity $v_2 - v_1$, the vertical hopping time when $y_2 - y_1 = 0$ can be recovered.

In [8]:
"""
Calculate the times at which horizontal hops occur
"""
function horizontal_hopping_times(times, positions, velocities)
    Δxs = [x[3] - x[1] for x in positions]  # x_2 - x_1
    Δus = [v[3] - v[1] for v in velocities]  # u_2 - u_1;
    
    # indices where there is a hop between collisions i and i+1: 
    horiz_hop_indices = find( sign(Δxs[i]) != sign(Δxs[i+1]) for i in 1:length(positions)-1 );  
    
    # x + t*u = 0   so   t = -x/u
    horiz_hopping_times = times[horiz_hop_indices] - (Δxs[horiz_hop_indices] ./ Δus[horiz_hop_indices])
    
    return horiz_hopping_times
end
## Verified by karel, it is correct.
 

"""
Calculate the times at which horizontal hops occur
"""
function vertical_hopping_times(times, positions, velocities)
    Δys = [x[4] - x[2] for x in positions]  # y_2 - y_1
    Δvs = [v[4] - v[2] for v in velocities]  # v_2 - v_1;
    
    # indices where there is a hop between collisions i and i+1: 
    vert_hop_indices = find( sign(Δys[i]) != sign(Δys[i+1]) for i in 1:length(positions)-1 );  
    
    # x + t*u = 0   so   t = -x/u
    vert_hopping_times = times[vert_hop_indices] - (Δys[vert_hop_indices] ./ Δvs[vert_hop_indices])
    
    return vert_hopping_times
end

vertical_hopping_times

 ## Analytical Expressions
 
 ### 1. Volume
 
 The Volume function has three regimenes. First, when all hopping is possible, that is, (convention $w>h$) $r<h/4$. Then, horizontal hopping is excluded, but vertical still posible, i.e. $h/4\leq r < w/4$. At last, no hopping is possible, that means, $w/4\leq r \leq r_{max}$. 
 
 The Excluded Volume is evaluated in two parts: the round part and the triangular part. We use the notation "Neg" in the functions to indicate that is the excluded volume, "free" to indicate all hoping is posible, "vhopp" to indicate only vertical hopping is possible, and "nohopp" to indicate no hopping is possible.

In [56]:
cte=3π/2 #E=1/2, -> abs(v^2)=1 as above
rhmax= h/4 #maximal radius in which a horizontal hopp is possible
rvmax= w/4 #maximal radius in which a vertical hopp is possible (any hopp is not posible in our geometry after this)
rmax=  (h+w-√(2*h*w))/2   #maximal possible radius for discs


volBox(a,b)=16*a^2*b^2
volNegfree(a,b,r)=16*π*a*b*r^2-64/3*r^3*(a+b)+8*r^4

expresioncircularvol(a,b,r,θ)=32*r^2*a*b*θ +64/3*r^3*(a*cos(θ)-b*sin(θ)) + 8*r^4*sin(θ)^2

function volNegCircle(a,b,r)
    # This function only has use if r>rhmax
    α=asin(b/r)
    if(r<rvmax)
        β=0
    else
        β=acos(a/r)
    end
    result=expresioncircularvol(a,b,r,α)-expresioncircularvol(a,b,r,β)
    return result
end

vollowertriangle(a,b,r)= 32*a^2*b*sqrt(r^2-a^2)-32/3*a^2*(r^2-a^2+2*b*sqrt(r^2-a^2))+8*a^2*(r^2-a^2)

voluppertriangle(a,b,r)= 32*b^2*a*sqrt(r^2-b^2)-32/3*b^2*(r^2-b^2+2*a*sqrt(r^2-b^2))+8*b^2*(r^2-b^2)


function volNegGeneral(a,b,r)
    if (r<rhmax)
        result=volNegfree(a,b,r)
    elseif (rhmax<=r) && (r<rvmax)
       # println("a=",a," r =" , r, "r^2-a^2 = ", r^2-a^2)
        result=volNegCircle(a,b,r)+voluppertriangle(a,b,r)
    else
        result=volNegCircle(a,b,r)+voluppertriangle(a,b,r)+vollowertriangle(a,b,r)
    end
    return result
end

function vol(a,b,r) 
    result=volBox(a,b)-volNegGeneral(a,b,r)
  #  print(" vol = ", result, " \t ")
    return result
end 

vol (generic function with 1 method)

## 2 Areas

#### Hopping Areas

In [57]:
area_vert_hopp(a,b,r)=16*√(2)*b*(r-a)^2 #This is right, but SymPy has problems refining Heaviside function and DiracDelta
area_horiz_hopp(a,b,r)=16*√(2)*a*(r-b)^2 # The Area HAS TWO SIDES. This is weird, but true.
#Those two don't generalize for larger r--- they just stop making sense.

area_horiz_hopp (generic function with 1 method)

### collision with wall
When hopping is no longer possible, the volume is split into disjoint components. This is equivalent to a factor
in the area of 2 or 4. Given that this only is relevant in this case, we take put those factors here. In the case of collition between discs, that factor is canceled as both the Volume and Area get split equally.

In [58]:
area_wall_free(a,b,r)=8*a*b^2-4*π*b*r^2+16/3*r^3 #perfect but needs generalization

exprpos_wall(a,b,r)=8*a*b^2
exprcircular_wall(a,b,r,θ)=8*b*r^2*θ+16/3*r^3*(cos(θ)) #sector

# la parte del "upper triangle"
a_wall_uppt(a,b,r)=8/3*b^2*sqrt(r^2-b^2)
a_wall_lowt(a,b,r)=8/3*a^2*sqrt(r^2-a^2)

function area_wall(a,b,r)
    if (r<rhmax)
        result=area_wall_free(a,b,r)
    elseif (rhmax<=r) && (r<rvmax)
         α=asin(b/r)
         negativo=exprcircular_wall(a,b,r,α)-exprcircular_wall(a,b,r,0)+a_wall_uppt(a,b,r)
         result=(exprpos_wall(a,b,r)-negativo)*2 # Actually is the Volume that gets split into disjoint componets
    else
        α=asin(b/r)
        β=acos(a/r)
        negativo=0 # (exprcircular_wall(a,b,r,α)-exprcircular_wall(a,b,r,β))+a_wall_uppt(a,b,r)+a_wall_lowt(a,b,r)
        result=(exprpos_wall(a,b,r)-negativo)*4
    end
    return result
end

area_wall (generic function with 1 method)

###  Collition between discs:
Although here the space get also split,the factor accounting for that does not appear as it cancels itself in Area and Volume.

In [62]:
area_col_free(a,b,r)=(16*π*a*b*r-32*(a+b)*r^2+16*r^3)*√(2) 
exprcircular_col(a,b,r,θ)=16*√2*(2*a*b*r*θ + 2*r^2*(a*cos(θ)-b*sin(θ)) + r^3*sin(θ)^2 )

function area_col(a,b,r)
    if (r<rhmax)
        result=area_col_free(a,b,r)
    elseif (rhmax<=r) && (r<rvmax)
     #   println(b, " ", r, " ", b/r)
        α=asin(b/r)
        result=exprcircular_col(a,b,r,α)-exprcircular_col(a,b,r,0)
    else
        α=asin(b/r)
        β=acos(a/r)        
        result=exprcircular_col(a,b,r,α)-exprcircular_col(a,b,r,β)   
    end
  #  print(" acol= ",result,  " \n ")
    return result
end

area_col (generic function with 1 method)

In [63]:
rango=(0.1, 0.2, 0.2501)
imagen=map(x->area_col(w/2-x,h/2-x,x),rango)

(1.3956909623144902, 0.9881971714694461, 0.4525529933634218)

In [64]:
w

1.5

### 3. Mean Times Machta Zwanzig

We combine area and volume with the general factor to obtain the theoretical mean times according to Machta Zwanzig Formula

In [65]:
horiz_hop_analytical(a,b,r)=cte*vol(a,b,r)/area_horiz_hopp(a,b,r)
vert_hop_analytical(a,b,r)=cte*vol(a,b,r)/area_vert_hopp(a,b,r)
right_wall_analytical(a,b,r)=cte*vol(a,b,r)/area_wall(a,b,r) #perfect
disc_collision_analytical(a,b,r)=cte*vol(a,b,r)/area_col(a,b,r)

horiz_hop_analytical_d(a, b, r) = (3π / (2*√(2))) * (2*a^2*b^2 - 2π*a*b*r^2 + ((a+b)/3)*(2*r)^3 - r^4) / (a*√(2)*(b-r)^2)
disc_collision_analytical_d(a, b, r) = (3π / (2*√(2))) * (2*a^2*b^2 - 2π*a*b*r^2 + ((a+b)/3)*(2*r)^3 - r^4) / (2π*a*b*r - 4*(a+b)*r^2 + 2r^3 )

disc_collision_analytical_d (generic function with 1 method)

The simulation function gives us a quick sanity check on the four event formulas above, but it doesn't give statistics. We relegate that to some other functions below.

In [66]:
function simulation(w, h, num_collisions=10^5)
    
    num_horiz_hop_data=Float64[]
    exact_horiz_hop_data=Float64[]
    
    num_vert_hop_data=Float64[]
    exact_vert_hop_data=Float64[]
    
    num_disc_collision_data = Float64[]
    exact_disc_collision_data = Float64[]
    
    num_rightwall_data = Float64[]
    exact_rightwall_data = Float64[]
    
    
    
    exhopd = Float64[]
    excold = Float64[]
    
    
    rs = 0.005:0.005:rmax #we do it until it cannot be done anymore
    
    for r in rs
        print(r, " ")
        
        a = w/2 - r
        b = h/2 - r
    
        times, positions, velocities, collision_types = dynamics(w, h, r, num_collisions);
        
        
        horiz_hopping_times = horizontal_hopping_times(times, positions, velocities)

        vert_hopping_times = vertical_hopping_times(times, positions, velocities)

        if(r<rhmax)
            push!(num_vert_hop_data, mean(diff(vert_hopping_times)))   # diff gives inter-hop times
            push!(exact_vert_hop_data, vert_hop_analytical(a, b, r))
        end
        
        push!(num_horiz_hop_data, mean(diff(horiz_hopping_times)))   # diff gives inter-hop times
        push!(exact_horiz_hop_data, horiz_hop_analytical(a, b, r))
        push!(exhopd, horiz_hop_analytical_d(a, b, r))
        
        
        disc_collision_times = times[collision_types .== 5]
        push!(num_disc_collision_data, mean(diff(disc_collision_times)))   # diff gives inter-hop times
        push!(exact_disc_collision_data, disc_collision_analytical(a, b, r))
        push!(excold, disc_collision_analytical_d(a, b, r))
    
        rightwall_times = times[collision_types .== 1]
        push!(num_rightwall_data, mean(diff(rightwall_times)))   # diff gives inter-hop times
        push!(exact_rightwall_data, right_wall_analytical(a, b, r))
        
    end
    
    return (rs,  num_vert_hop_data, exact_vert_hop_data, num_horiz_hop_data, exact_horiz_hop_data, num_disc_collision_data, 
        exact_disc_collision_data, num_rightwall_data, exact_rightwall_data, excold, exhopd)
                
    end

simulation (generic function with 2 methods)

In [67]:
w, h = 1.5, 1.0
nevents=10^6
(rs, num_vert_hop_data, exact_vert_hop_data,  num_horiz_hop_data, exact_horiz_hop_data, num_disc_collision_data, 
    exact_disc_collision_data, num_rightwall_data, exact_rightwalldata, excold, exhopd)= simulation(w, h, nevents);

0.005 0.01 0.015 0.02 0.025 0.03 0.035 0.04 0.045 0.05 0.055 0.06 0.065 0.07 0.075 0.08 0.085 0.09 0.095 0.1 0.105 0.11 0.115 0.12 0.125 0.13 0.135 0.14 0.145 0.15 0.155 0.16 0.165 0.17 0.17500000000000002 0.18 0.185 0.19 0.195 0.2 0.20500000000000002 0.21 0.215 0.22 0.225 0.23 0.23500000000000001 0.24 0.245 0.25 0.255 0.26 0.265 0.27 0.275 0.28 0.28500000000000003 0.29 0.295 0.3 0.305 0.31 0.315 0.32 0.325 0.33 0.335 0.34 0.34500000000000003 0.35000000000000003 0.355 0.36 0.365 0.37 0.375 0.38 

In [17]:
using JLD # To save the data, and save us headeaches

In [18]:
save("DatosNumericos10a6.jld", "rs", rs, 
    "num_vert_hop_data", num_vert_hop_data,
    "num_horiz_hop_data", num_horiz_hop_data,
    "num_disc_collision_data", num_disc_collision_data,
    "num_rightwall_data", num_rightwall_data
)

In [19]:
Plots.scatter(rs, num_rightwall_data, m=:square, label="numerical") # super
plot!(rs, exact_rightwalldata, label="exact", linewidth=2) #absolut PERFECT

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 0.1 
 
 
 0.2 
 
 
 0.3 
 
 
 0 
 
 
 2 
 
 
 4 
 
 
 6 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<polyline clip-path="url(#clip2002)" style="stroke:#e26f46; stroke-width:2; stroke-opacity:1; fill:none" points="
 38.1295,23.5949 45.1481,25.9981 52.1666,28.4735 59.1852,31.0213 66.2038,33.6416 73.2224,36.3346 80.241,39.1007 87.2595,41.9402 94.2781,44.8533 101.297,47.8407 
 108.315,50.9028 115.334,54.0401 122.352,57.2533 129.371,60.5431 136.39,63.9103 143.408,67.3558 150.427,70.8805 157.445,74.4855 164.464,78.1718 171.483,81.9408 
 178.501,85.7936 185.52,89.7319 192.538,93.757 199.557,97.8706 206.575,102.074 213.594,106.37 220.613,110.76 227.631,115.246 234.65,119.83 241.668,124.514 
 248.687,129.3 255.706,134.191 262.724,139.189 269.743,144.295 276.761,149.512 283.78,154.84 290.798,160.28 297.817,165.834 304.836,171.499 311.854,177.274 
 318.873,183.155 325.891,189.136 332.91,195.206 339.929,201.352 346.947,207.554 353.966,213.787 360.984,220.013 368.003,226.186 375.021,232.242 382.04,301.805 
 389.059,304.618 396.077,307.363 403.096,310.059 410.114,312.716 417.133,315.341 424.152,317.939 431.17,320.514 438.189,323.067 445.207,325.601 452.226,328.119 
 459.244,330.621 466.263,333.11 473.282,335.585 480.3,338.048 487.319,340.5 494.337,342.942 501.356,345.373 508.374,347.795 515.393,350.206 522.412,352.607 
 529.43,354.996 536.449,357.368 543.467,359.713 550.486,361.982 557.505,365.503 564.523,365.511 
 "/>
 
 
 
 
 
 numerical 
 
 
 
 exact

In [20]:
Plots.scatter(rs, num_disc_collision_data, m=:square, label="numerical")
plot!(rs, exact_disc_collision_data, label="exact")
plot!(rs, excold, label="DavEx") #DFormula that doesn't account for the discontinuites
plot!(rs, zeros(rs), lw=2, label="vertical zero line")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 0.1 
 
 
 0.2 
 
 
 0.3 
 
 
 -25 
 
 
 0 
 
 
 25 
 
 
 50 
 
 
 75 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<polyline clip-path="url(#clip2202)" style="stroke:#e26f46; stroke-width:1; stroke-opacity:1; fill:none" points="
 51.2582,22.6535 58.1068,156.105 64.9555,200.612 71.8041,222.884 78.6527,236.262 85.5013,245.192 92.35,251.581 99.1986,256.382 106.047,260.124 112.896,263.125 
 119.745,265.586 126.593,267.643 133.442,269.389 140.29,270.89 147.139,272.196 153.988,273.343 160.836,274.359 167.685,275.266 174.534,276.08 181.382,276.817 
 188.231,277.486 195.079,278.097 201.928,278.658 208.777,279.175 215.625,279.652 222.474,280.095 229.323,280.507 236.171,280.892 243.02,281.251 249.869,281.588 
 256.717,281.905 263.566,282.203 270.414,282.483 277.263,282.748 284.112,282.998 290.96,283.234 297.809,283.457 304.658,283.668 311.506,283.867 318.355,284.054 
 325.203,284.23 332.052,284.394 338.901,284.547 345.749,284.688 352.598,284.816 359.447,284.929 366.295,285.027 373.144,285.104 379.992,285.158 386.841,285.181 
 393.69,285.23 400.538,285.303 407.387,285.386 414.236,285.477 421.084,285.573 427.933,285.672 434.782,285.775 441.63,285.879 448.479,285.985 455.327,286.093 
 462.176,286.203 469.025,286.313 475.873,286.424 482.722,286.535 489.571,286.648 496.419,286.76 503.268,286.873 510.116,286.987 516.965,287.1 523.814,287.214 
 530.662,287.327 537.511,287.44 544.36,287.552 551.208,287.661 558.057,287.738 564.906,287.782 
 "/>
<polyline clip-path="url(#clip2202)" style="stroke:#3da44d; stroke-width:1; stroke-opacity:1; fill:none" points="
 51.2582,22.6535 58.1068,156.105 64.9555,200.612 71.8041,222.884 78.6527,236.262 85.5013,245.192 92.35,251.581 99.1986,256.382 106.047,260.124 112.896,263.125 
 119.745,265.586 126.593,267.643 133.442,269.389 140.29,270.89 147.139,272.196 153.988,273.343 160.836,274.359 167.685,275.266 174.534,276.08 181.382,276.817 
 188.231,277.486 195.079,278.097 201.928,278.658 208.777,279.175 215.625,279.652 222.474,280.095 229.323,280.507 236.171,280.892 243.02,281.251 249.869,281.588 
 256.717,281.905 263.566,282.203 270.414,282.483 277.263,282.748 284.112,282.998 290.96,283.234 297.809,283.457 304.658,283.668 311.506,283.867 318.355,284.054 
 325.203,284.23 332.052,284.394 338.901,284.547 345.749,284.688 352.598,284.816 359.447,284.929 366.295,285.027 373.144,285.104 379.992,285.158 386.841,285.181 
 393.69,285.161 400.538,285.079 407.387,284.899 414.236,284.549 421.084,283.854 427.933,282.268 434.782,276.768 441.63,365.511 448.479,295.958 455.327,292.155 
 462.176,290.871 469.025,290.271 475.873,289.952 482.722,289.778 489.571,289.686 496.419,289.646 503.268,289.641 510.116,289.66 516.965,289.698 523.814,289.75 
 530.662,289.813 537.511,289.884 544.36,289.964 551.208,290.05 558.057,290.143 564.906,290.241 
 "/>
<polyline clip-path="url(#clip2202)" style="stroke:#c271d2; stroke-width:2; stroke-opacity:1; fill:none" points="
 51.2582,287.831 58.1068,287.831 64.9555,287.831 71.8041,287.831 78.6527,287.831 85.5013,287.831 92.35,287.831 99.1986,287.831 106.047,287.831 112.896,287.831 
 119.745,287.831 126.593,287.831 133.442,287.831 140.29,287.831 147.139,287.831 153.988,287.831 160.836,287.831 167.685,287.831 174.534,287.831 181.382,287.831 
 188.231,287.831 195.079,287.831 201.928,287.831 208.777,287.831 215.625,287.831 222.474,287.831 229.323,287.831 236.171,287.831 243.02,287.831 249.869,287.831 
 256.717,287.831 263.566,287.831 270.414,287.831 277.263,287.831 284.112,287.831 290.96,287.831 297.809,287.831 304.658,287.831 311.506,287.831 318.355,287.831 
 325.203,287.831 332.052,287.831 338.901,287.831 345.749,287.831 35

In [21]:
rhmax, rvmax, rmax

(0.25, 0.375, 0.3839745962155614)

In [22]:
#rsredux=rs[find( r<rhmax ) in rs]
rsredux=filter(r->r<rhmax, rs)
plot(rsredux, num_vert_hop_data, m=:square, label="numerical")
plot!(rsredux, exact_vert_hop_data, label="exact") 
#plot!(rs, exhopd, label="DavidEx")
#Formula David, raiz de dos muy grande 

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.00 
 
 
 0.05 
 
 
 0.10 
 
 
 0.15 
 
 
 0.20 
 
 
 0.25 
 
 
 1.1 
 
 
 1.2 
 
 
 1.3 
 
 
 1.4 
 
 
 1.5 
 
 
 1.6 
 
 
 1.7 
 
<polyline clip-path="url(#clip2402)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 47.7342,39.1053 58.5064,36.464 69.2787,39.607 80.051,35.2251 90.8232,30.7526 101.595,30.0151 112.368,25.6492 123.14,24.7344 133.912,26.99 144.685,27.9266 
 155.457,25.7301 166.229,22.113 177.001,30.0165 187.774,25.6157 198.546,30.2666 209.318,31.485 220.09,29.6276 230.863,34.6601 241.635,38.0559 252.407,42.9133 
 263.179,43.721 273.952,47.8892 284.724,56.1169 295.496,50.4051 306.269,60.154 317.041,63.3286 327.813,74.6113 338.585,78.3624 349.358,90.077 360.13,95.4646 
 370.902,102.172 381.674,118.639 392.447,123.403 403.219,133.771 413.991,146.801 424.763,154.975 435.536,166.315 446.308,182.159 457.08,197.278 467.853,209.815 
 478.625,229.953 489.397,245.365 500.169,261.276 510.942,275.626 521.714,291.694 532.486,315.596 543.258,332.291 554.031,347.213 564.803,364.056 
 "/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<polyline clip-path="url(#clip2402)" style="stroke:#e26f46; stroke-width:1; stroke-opacity:1; fill:none" points="
 47.7342,40.4496 58.5064,37.8936 69.2787,35.5606 80.051,33.4584 90.8232,31.5953 101.595,29.98 112.368,28.6213 123.14,27.5287 133.912,26.712 144.685,26.1814 
 155.457,25.9476 166.229,26.0216 177.001,26.4152 187.774,27.1404 198.546,28.2099 209.318,29.6367 220.09,31.4347 230.863,33.618 241.635,36.2015 252.407,39.2005 
 263.179,42.6311 273.952,46.5097 284.724,50.8535 295.496,55.6799 306.269,61.0071 317.041,66.8536 327.813,73.2383 338.585,80.1801 349.358,87.6984 360.13,95.8123 
 370.902,104.541 381.674,113.901 392.447,123.912 403.219,134.588 413.991,145.944 424.763,157.99 435.536,170.733 446.308,184.176 457.08,198.312 467.853,213.13 
 478.625,228.604 489.397,244.694 500.169,261.342 510.942,278.465 521.714,295.95 532.486,313.641 543.258,331.331 554.031,348.743 564.803,365.511 
 "/>
 
 
 
 
 
 
 numerical 
 
 
 
 exact

## Statistics.

Some Plots of interesting statistical propiertes, such as histograms.

In [23]:
using StatsBase

In [69]:
radiosinteresantes=(0.01, 0.075, 0.125, 0.15, 0.2, 0.24, 0.249)
thopforhisto=Dict{Float64, Array}()

binlim=10
paso=0.5
rangobins=0:paso:binlim
rangoplot=rangobins[1:end-1]+paso/2

hth=Dict{Float64, Any}()

for r in radiosinteresantes
    w,h=1.5,1
    num_collisions=500000
    times, positions, velocities, collision_types = dynamics(w, h, r, num_collisions);
    thopforhisto[r] = horizontal_hopping_times(times, positions, velocities);
    #normalized histograms
    aux=fit(Histogram, diff(thopforhisto[r]), rangobins, closed=:left)
    hth[r]=aux.weights/sum(aux.weights)
end

In [70]:
p=plot(xlim=(0,binlim), xlabel="Time of hop", title="Freq")
for r in radiosinteresantes
    plot!(rangoplot,hth[r], marker=(:rect, 3, :match, stroke(0,0, :match, :line)), lc=:match, label="$r")
end
savefig("histogramahorizontalhopsemidecente01.png")
p

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 2 
 
 
 4 
 
 
 6 
 
 
 8 
 
 
 10 
 
 
 0.0 
 
 
 0.1 
 
 
 0.2 
 
 
 0.3 
 
 
 Freq 
 
 
 Time of hop 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.01 
 
 
 
 
 0.075 
 
 
 
 
 0.125 
 
 
 
 
 0.15 
 
 
 
 
 0.2 
 
 
 
 
 0.24 
 
 
 
 
 0.249

In [73]:
p=0

0

# Hitting right wall

Simulation to obtain statistics of collision of disk 1 with the right wall.

In [74]:
function simulationhitwall(w, h, num_collisions=10^5)
    
    num_hitting_times = Float64[]
    exact_hitting_times = Float64[]
    σ=Float64[]
    stderror=Float64[]
    
    rs = 0.0051:0.00256:rmax
    
    for r in rs
        print(r, " ")
        
        a = w/2 - r
        b = h/2 - r
    
        nhits=0
        disc_1_hits_right_wall_times=[]
        while(nhits<200)
            times, positions, velocities, collision_types = dynamics(w, h, r, num_collisions); 
            disc_1_hits_right_wall_times = diff(times[collision_types .== 1])
            nhits=length(disc_1_hits_right_wall_times)
   #         println(r, " ", nhits, "tu abuela", disc_1_hits_right_wall_times[1])
        end
        
        auxhit=mean(disc_1_hits_right_wall_times)
   #     println(auxhit)
        push!(num_hitting_times, auxhit)
        
        push!(exact_hitting_times, right_wall_analytical(a, b, r))
        aux=std(disc_1_hits_right_wall_times)
        push!(σ, aux)
        push!(stderror, aux/sqrt(nhits))
        
        
    end
    
    return rs, num_hitting_times, exact_hitting_times, σ, stderror
end

simulationhitwall (generic function with 2 methods)

In [75]:
rs, num_hitting_times, exact_hitting_times, stdeviation, stderror =simulationhitwall(1.5, 1.0, nevents);

0.0051 0.00766 0.01022 0.012780000000000001 0.015340000000000001 0.017900000000000003 0.020460000000000002 0.023020000000000002 0.025580000000000002 0.028140000000000002 0.0307 0.033260000000000005 0.035820000000000005 0.038380000000000004 0.040940000000000004 0.043500000000000004 0.046060000000000004 0.048620000000000003 0.05118 0.05374 0.0563 0.05886 0.06142 0.06398000000000001 0.06654 0.06910000000000001 0.07166 0.07422000000000001 0.07678 0.07934000000000001 0.0819 0.08446000000000001 0.08702000000000001 0.08958 0.09214000000000001 0.0947 0.09726000000000001 0.09982 0.10238000000000001 0.10494 0.10750000000000001 0.11006 0.11262000000000001 0.11518 0.11774000000000001 0.1203 0.12286000000000001 0.12542 0.12798 0.13054000000000002 0.13310000000000002 0.13566 0.13822 0.14078000000000002 0.14334000000000002 0.1459 0.14846 0.15102000000000002 0.15358000000000002 0.15614 0.1587 0.16126000000000001 0.16382000000000002 0.16638 0.16894 0.1715 0.17406000000000002 0.17662000000000003 0.17918

In [76]:
p=plot(xlim=(0,rmax), title="Mean Time for Right wall Collision", xlabel="radius",ylabel="time" )
scatter!(rs, num_hitting_times, m=:square, ms=2, label="numerical", yerror=stderror)
plot!(rs, exact_hitting_times, label="exact", lw=3)
savefig("HopTimesWithError01.png")
p

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 0.1 
 
 
 0.2 
 
 
 0.3 
 
 
 0 
 
 
 2 
 
 
 4 
 
 
 6 
 
 
 Mean Time for Right wall Collision 
 
 
 radius 
 
 
 time 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<polyline clip-path="url(#clip3902)" style="stroke:#e26f46; stroke-width:3; stroke-opacity:1; fill:none" points="
 45.2877,40.7952 48.9028,41.9033 52.5179,43.0286 56.133,44.171 59.7481,45.3307 63.3632,46.5075 66.9783,47.7015 70.5934,48.9128 74.2085,50.1414 77.8236,51.3872 
 81.4387,52.6504 85.0538,53.931 88.6689,55.2289 92.284,56.5443 95.8991,57.8772 99.5142,59.2277 103.129,60.5957 106.744,61.9814 110.359,63.3849 113.975,64.8061 
 117.59,66.2451 121.205,67.7021 124.82,69.1771 128.435,70.6701 132.05,72.1814 135.665,73.7109 139.28,75.2588 142.895,76.8251 146.51,78.41 150.126,80.0135 
 153.741,81.6359 157.356,83.2772 160.971,84.9375 164.586,86.617 168.201,88.3159 171.816,90.0342 175.431,91.7721 179.046,93.5298 182.661,95.3074 186.277,97.1051 
 189.892,98.9231 193.507,100.762 197.122,102.621 200.737,104.501 204.352,106.402 207.967,108.324 211.582,110.268 215.197,112.233 218.812,114.22 222.428,116.23 
 226.043,118.262 229.658,120.316 233.273,122.394 236.888,124.494 240.503,126.618 244.118,128.766 247.733,130.937 251.348,133.133 254.964,135.353 258.579,137.597 
 262.194,139.867 265.809,142.162 269.424,144.483 273.039,146.829 276.654,149.201 280.269,151.599 283.884,154.023 287.499,156.474 291.115,158.952 294.73,161.456 
 298.345,163.987 301.96,166.545 305.575,169.13 309.19,171.742 312.805,174.379 316.42,177.044 320.035,179.733 323.65,182.448 327.266,185.188 330.881,187.952 
 334.496,190.738 338.111,193.545 341.726,196.373 345.341,199.219 348.956,202.08 352.571,204.955 356.186,207.841 359.801,210.733 363.417,213.628 367.032,216.521 
 370.647,219.407 374.262,222.278 377.877,225.129 381.492,227.949 385.107,230.731 388.722,233.462 392.337,293.448 395.952,294.756 399.568,296.046 403.183,297.321 
 406.798,298.584 410.413,299.836 414.028,301.078 417.643,302.312 421.258,303.537 424.873,304.756 428.488,305.967 432.103,307.173 435.719,308.372 439.334,309.567 
 442.949,310.756 446.564,311.94 450.179,313.12 453.794,314.296 457.409,315.467 461.024,316.635 464.639,317.799 468.254,318.959 471.87,320.117 475.485,321.27 
 479.1,322.421 482.715,323.569 486.33,324.714 489.945,325.856 493.56,326.996 497.175,328.133 500.79,329.268 504.405,330.4 508.021,331.529 511.636,332.656 
 515.251,333.781 518.866,334.904 522.481,336.024 526.096,337.142 529.711,338.257 533

# Disc collisions

In [77]:
function simulation_disc_collisions(w, h, num_collisions=10^5)
    
    num_times = Float64[]
    exact_times = Float64[]
    σ=Float64[]
    stderror=Float64[]
    
    rs = 0.0051:0.005:rmax
    
    for r in rs
        print(r, " ")
        
        a = w/2 - r
        b = h/2 - r
    
        times, positions, velocities, collision_types = dynamics(w, h, r, num_collisions);
        
        collision_times = diff(times[collision_types .== 5])
        nhits=length(collision_times)
        
        push!(num_times, mean(collision_times))   # diff gives inter-hop times
        push!(exact_times, disc_collision_analytical(a, b, r))
        aux=std(num_times)
        push!(σ, aux)
        push!(stderror, aux/sqrt(nhits))
        
        
    end
    
    return rs, num_times, exact_times, σ, stderror
end

simulation_disc_collisions (generic function with 2 methods)

In [84]:
rs, num_disc_collision_times, exact_disc_collision_times, colsig, colerror = simulation_disc_collisions(w, h, nevents)

0.0051 0.010100000000000001 0.0151 0.0201 0.0251 0.030100000000000002 0.0351 0.040100000000000004 0.0451 0.0501 0.0551 0.0601 0.0651 0.0701 0.0751 0.0801 0.08510000000000001 0.0901 0.0951 0.10010000000000001 0.1051 0.1101 0.11510000000000001 0.1201 0.1251 0.1301 0.1351 0.1401 0.1451 0.1501 0.15510000000000002 0.1601 0.1651 0.1701 0.1751 0.1801 0.18510000000000001 0.1901 0.1951 0.2001 0.2051 0.2101 0.2151 0.22010000000000002 0.2251 0.2301 0.2351 0.2401 0.2451 0.2501 0.2551 0.2601 0.2651 0.2701 0.2751 0.2801 0.2851 0.2901 0.29510000000000003 0.30010000000000003 0.3051 0.3101 0.3151 0.3201 0.3251 0.3301 0.3351 0.3401 0.3451 0.3501 0.3551 0.36010000000000003 0.3651 0.3701 0.3751 0.3801 

(0.0051:0.005:0.3801, [78.2915, 38.9922, 26.036, 19.2218, 15.3037, 12.6646, 10.7887, 9.39099, 8.22668, 7.33084  …  0.284872, 0.251711, 0.21681, 0.184504, 0.149909, 0.116044, 0.0825195, 0.0499065, 0.027966, 0.014004], [77.4969, 38.8773, 25.8264, 19.2632, 15.3105, 12.6674, 10.7743, 9.35066, 8.24032, 7.34949  …  0.284737, 0.250957, 0.217127, 0.183278, 0.149453, 0.115729, 0.0822939, 0.0498657, 0.0277059, 0.0140215], [NaN, 27.7888, 27.212, 26.4094, 25.5232, 24.6694, 23.8732, 23.1385, 22.4701, 21.8559  …  11.0925, 11.0239, 10.9568, 10.8909, 10.8264, 10.7632, 10.7012, 10.6404, 10.5806, 10.5219], [NaN, 0.210292, 0.169866, 0.142704, 0.124254, 0.11027, 0.0994043, 0.0907825, 0.0832861, 0.0772772  …  0.0199884, 0.019671, 0.0192835, 0.0189549, 0.0185803, 0.0181881, 0.0177907, 0.0173923, 0.0173082, 0.0174337])

In [85]:
p=plot(xlim=(0,rmax), title="Mean Time Discs Collisions", xlabel="radius",ylabel="time" )
scatter!(rs, num_disc_collision_times, m=:square, ms=2, label="numerical", yerror=colerror)
plot!(rs, exact_disc_collision_times, label="exact", lw=2)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 0.1 
 
 
 0.2 
 
 
 0.3 
 
 
 0 
 
 
 20 
 
 
 40 
 
 
 60 
 
 
 80 
 
 
 Mean Time Discs Collisions 
 
 
 radius 
 
 
 time 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<polyline clip-path="url(#clip5102)" style="stroke:#e26f46; stroke-width:2; stroke-opacity:1; fill:none" points="
 51.7898,43.9412 58.7647,196.837 65.7396,248.505 72.7146,274.489 79.6895,290.138 86.6644,300.602 93.6394,308.097 100.614,313.733 107.589,318.129 114.564,321.656 
 121.539,324.55 128.514,326.97 135.489,329.024 142.464,330.791 149.439,332.328 156.414,333.678 163.389,334.875 170.364,335.942 177.339,336.902 184.314,337.77 
 191.288,338.558 198.263,339.278 205.238,339.939 212.213,340.548 219.188,341.111 226.163,341.633 233.138,342.119 240.113,342.572 247.088,342.995 254.063,343.393 
 261.038,343.766 268.013,344.117 274.988,344.448 281.963,344.76 288.938,345.054 295.913,345.333 302.887,345.596 309.862,345.844 316.837,346.079 323.812,346.299 
 330.787,346.507 337.762,346.701 344.737,346.881 351.712,347.047 358.687,347.197 365.662,347.331 372.637,347.445 379.612,347.537 386.587,347.599 393.562,347.626 
 400.537,347.684 407.511,347.771 414.486,347.87 421.461,347.977 428.436,348.09 435.411,348.208 442.386,348.328 449.361,348.452 456.336,348.577 463.311,348.705 
 470.286,348.834 477.261,348.964 484.236,349.095 491.211,349.227 498.186,349.359 505.161,349.492 512.136,349.625 519.11,349.759 526.085,349.893 533.06,350.027 
 540.035,350.161 547.01,350.295 553.985,350.427 560.96,350.555 567.935,350.643 574.91,350.697 
 "/>
 
 
 
 
 
 numerical 
 
 
 
 exact

Something to do with relative velocity?

## Vertical hopping

In [125]:
function simulation_vert_hopping(w, h, num_collisions=10^5)
    
    num_times = Float64[]
    exact_times = Float64[]
    
     σ=Float64[]
    stderror=Float64[]
    
    
    rs = 0.005:0.01:rvmax
    
    for r in rs
        print(r, " ")
    
        a = w/2 - r
        b = h/2 - r
    
        vert_hopping_times=[]
        nhits=0
    
    #    while(nhits<200)
            times, positions, velocities, collision_types = dynamics(w, h, r, num_collisions);
            vert_hopping_times = diff(vertical_hopping_times(times, positions, velocities))
            nhits=length(vert_hopping_times)
#        end
             
        push!(num_times, mean(vert_hopping_times))   # diff gives inter-hop times
        push!(exact_times, vert_hop_analytical(a, b, r))
        
        aux=std(vert_hopping_times)
        push!(σ, aux)
        push!(stderror, aux/sqrt(nhits))
       
    end

    return rs, num_times,exact_times, σ, stderror


end

simulation_vert_hopping (generic function with 2 methods)

In [136]:
rs, num_vert_hopping_times, exact_vert_hopping_times , vhsigma, vhoperror = simulation_vert_hopping(w, h, nevents)

0.005 0.015 0.025 0.035 0.045 0.055 0.065 0.075 0.085 0.095 0.105 0.115 0.125 0.135 0.145 0.155 0.165 0.175 0.185 0.195 0.205 0.215 0.225 0.235 0.245 0.255 0.265 0.275 0.285 0.295 0.305 0.315 0.325 0.335 0.345 0.355 0.365 0.375 

(0.005:0.01:0.375, [1.67754, 1.6852, 1.69105, 1.69149, 1.70978, 1.69563, 1.70556, 1.69287, 1.69829, 1.67679  …  0.846559, 0.802014, 0.75854, 0.722952, 0.688267, 0.656171, 0.63562, 0.623519, 0.680629, NaN], [1.67143, 1.68087, 1.68853, 1.69427, 1.69795, 1.69943, 1.69853, 1.69506, 1.68884, 1.67963  …  0.842982, 0.800626, 0.760385, 0.722354, 0.686986, 0.655452, 0.630805, 0.622978, 0.688219, Inf], [1.29909, 1.34559, 1.42916, 1.49966, 1.50217, 1.47903, 1.51924, 1.51229, 1.54561, 1.55004  …  0.728997, 0.695367, 0.669279, 0.645838, 0.633985, 0.630271, 0.651767, 0.696261, 0.911338, NaN], [0.00201715, 0.00213418, 0.00231318, 0.00247183, 0.00253708, 0.00253816, 0.00266813, 0.00270118, 0.00282576, 0.00287757  …  0.00166014, 0.0016108, 0.00158361, 0.00158564, 0.00162728, 0.00172267, 0.00196483, 0.00243652, 0.00433107, NaN])

In [138]:
p=plot(xlim=(0,rvmax), ylim=(0,2),title="Mean Time, Vertical Hop", xlabel="radius",ylabel="time" )
scatter!(rs, num_vert_hopping_times, m=:square, ms=2, label="numerical", yerror=vhoperror)
plot!(rs, exact_vert_hopping_times, label="exact", lw=2)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 0.1 
 
 
 0.2 
 
 
 0.3 
 
 
 0.0 
 
 
 0.5 
 
 
 1.0 
 
 
 1.5 
 
 
 2.0 
 
 
 Mean Time, Vertical Hop 
 
 
 radius 
 
 
 time 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<polyline clip-path="url(#clip7602)" style="stroke:#e26f46; stroke-width:2; stroke-opacity:1; fill:none" points="
 55.0681,85.4748 69.2639,83.9242 83.4598,82.6665 97.6557,81.7232 111.852,81.1176 126.047,80.8751 140.243,81.0235 154.439,81.5927 168.635,82.6155 182.831,84.1274 
 197.027,86.1667 211.223,88.7747 225.418,91.9952 239.614,95.8746 253.81,100.461 268.006,105.803 282.202,111.947 296.398,118.935 310.594,126.798 324.789,135.545 
 338.985,145.153 353.181,155.536 367.377,166.513 381.573,177.735 395.769,188.576 409.965,198.077 424.16,206.464 438.356,214.244 452.552,221.576 466.748,228.535 
 480.944,235.146 495.14,241.394 509.336,247.204 523.532,252.385 537.727,256.434 551.923,257.72 566.119,247.002 
 "/>
 
 
 
 
 
 numerical 
 
 
 
 exact